In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import sys
import pandas as pd
from astropy.table import Table, Column, vstack, join, hstack
import csv
import random


import matplotlib as mpl
mpl.rcParams['lines.linewidth']   = 1.5
mpl.rcParams['axes.linewidth']    = 1.5
mpl.rcParams['xtick.major.width'] =1.5
mpl.rcParams['ytick.major.width'] =1.5
mpl.rcParams['ytick.labelsize'] = 10
mpl.rcParams['xtick.labelsize'] = 10
mpl.rcParams['axes.labelsize'] = 11
mpl.rcParams['legend.numpoints'] = 1
mpl.rcParams['axes.labelweight']='semibold'
mpl.rcParams['font.weight'] = 'semibold'
mpl.rcParams['axes.titlesize']=9

In [ ]:
chains = np.load('./LCs/emceechain-56658270.npy')
prob = np.load('./LCs/emceeprob-56658270.npy')
probs=prob[:,50000:].reshape((-1))

In [ ]:
photModel = []
with open(f'./LCs/photmodel_TIC_56658270.txt','r') as inpt:
    rd = csv.reader(inpt, delimiter = ',')
    for r in rd:
        photModel.append(r)

In [ ]:
file = f"./LCs/TESSfit.pkl"
with open(file, 'rb') as fl:
    mydict = pickle.load(fl)

data = mydict['0']['0']['0']


In [ ]:
time = []
phased = []
flux = []
fluxError = []
mu = []
pmodel = []
model = []

for p in photModel:
    p = p[0].split(' ')
    time.append(float(p[0]))
    phased.append(float(p[1]))
    flux.append(float(p[2]))
    fluxError.append(float(p[3]))
    mu.append(float(p[4])) #gp 
    pmodel.append(float(p[5])) #transit model
    model.append(float(p[4]) + float(p[5])) #gp with transit model
    
p = np.asarray(p)
time = np.asarray(time)
phased = np.asarray(phased)
flux = np.asarray(flux)
fluxError = np.asarray(fluxError)
mu = np.asarray(mu)
pmodel = np.asarray(pmodel)
model = np.asarray(model) 

T0 = mydict['epoch1'][0]
Period = mydict['Per1'][0]
tt = (time % Period)-(T0 % Period)
l = np.argsort(tt)

In [ ]:
phase = data['phase']
y_gp = data['y_gp']
transit_model = data['transit_model']

In [ ]:
x = tt[l]
y = flux[l] - mu[l]

xbin = []
ybin = []
ystd = []
start = x[0]

binmin = 20

i = 0
go = True
while (x[i] < .74) and go:
    thisBuckety = []
    thisBucketx = []
    try:
        while x[i] < (start + binmin/60/24):
            try:
                thisBuckety.append(y[i])
                thisBucketx.append(x[i])
                i += 1
            except:
                start += binmin/60/24
                go = False
                break

        if len(thisBuckety) != 0:
            xbin.append(np.nanmedian(thisBucketx))
            ybin.append(np.nanmedian(thisBuckety))
            ystd.append(np.nanstd(thisBuckety))
        start += binmin/60/24
    except:
        break
    
    
xbin = np.asarray(xbin)
ybin = np.asarray(ybin)
yerr = np.asarray(ystd) / (np.sqrt(len(ystd)))

In [ ]:
import batman

In [ ]:
plt.rcParams['ytick.labelsize'] = 30
plt.rcParams['xtick.labelsize'] = 30

plt.rcParams['axes.linewidth']    = 4

plt.rcParams['xtick.minor.width'] = 2
plt.rcParams['ytick.minor.width'] = 2

plt.rcParams['xtick.minor.size'] = 5
plt.rcParams['ytick.minor.size'] = 4 

plt.rcParams['xtick.major.size'] = 6
plt.rcParams['ytick.major.size'] = 5

In [ ]:
burnin = 50000
samples=chains[:,burnin:,:].reshape((-1,12))

In [ ]:
l = np.squeeze((np.where(probs == np.max(probs))))[1] 
best = samples[l,:]

logP = best[7] #np.median(samples[:,7])
logamp = best[8] #np.median(samples[:,8])
logQ1 = best[9] #np.median(samples[:,9])
logQ2 = best[10] #np.median(samples[:,10])
logmix = best[11] #np.median(samples[:,11])

t0 = best[0] #np.median(samples[:,0])
per = best[1] #np.median(samples[:,1])
rprs = best[2] #np.median(samples[:,2])
b = best[3] #np.median(samples[:,3])
rho = best[4] #np.median(samples[:,4])
q1 = best[5] #np.median(samples[:,5])
q2 = best[6] #np.median(samples[:,6])
w = 0.
ecc = 0.0

masterParameters = [t0, per, rprs, b, rho, q1,q2,0,0]

In [ ]:
plt.figure(figsize = (10,8))
t0 = 0 
per = 1
rprstar = 2
b = 3
rhostar = 4
q1 = 5
q2 = 6
sqrtesinw = 7
sqrtecosw = 8


j = -1

burnin = 50000

for i in np.random.randint(0, high=150, size=50):
    
    medianChain = chains[i][j]
    
    ecc = 0 
    omega = 90 
    g1 = 2.0*medianChain[q2]*np.sqrt(medianChain[q1])
    g2 = np.sqrt(medianChain[q1])*(1.0-2.0*medianChain[q2])
    

    params = batman.TransitParams()
    params.t0 = 0 
    params.per = medianChain[per]                         
    params.rp = medianChain[rprstar]                    
    params.a = 215.*medianChain[rhostar]**(1./3.)*(medianChain[per]/365.25)**(2./3.)*((1.+ecc*np.sin(omega*np.pi/180.))/np.sqrt(1.-ecc**2))  #semi-major axis (in units of stellar radii) #(chains[i][-1][per] / 365)**(2/3) * 215.032 #
    params.inc = np.arccos(medianChain[b]/params.a/(1.0-ecc**2))*180./np.pi                 
    params.ecc = ecc                    
    params.w = omega                       
    params.u = [g1, g2]                
    params.limb_dark = "quadratic"     

    t = np.linspace(-.4, .4, 10000)
    m = batman.TransitModel(params, t)  
    f = m.light_curve(params)        

    
    plt.plot(t*24, f, alpha = .15, color = 'maroon', linewidth = 2)


## Best fit Transit
medianChain = masterParameters
    
ecc = 0
omega = 90
g1 = 2.0*medianChain[q2]*np.sqrt(medianChain[q1])
g2 = np.sqrt(medianChain[q1])*(1.0-2.0*medianChain[q2])


params = batman.TransitParams()
params.t0 = 0 
params.per = medianChain[per]                          
params.rp = medianChain[rprstar]                     
params.a = 215.*medianChain[rhostar]**(1./3.)*(medianChain[per]/365.25)**(2./3.)*((1.+ecc*np.sin(omega*np.pi/180.))/np.sqrt(1.-ecc**2))  #semi-major axis (in units of stellar radii) #(chains[i][-1][per] / 365)**(2/3) * 215.032 #
params.inc = np.arccos(medianChain[b]/params.a/(1.0-ecc**2))*180./np.pi            
params.ecc = ecc                  
params.w = omega                    
params.u = [g1, g2]               
params.limb_dark = "quadratic"     


t = np.linspace(-.4, .4, 10000)
m = batman.TransitModel(params, t) 
f = m.light_curve(params)         


plt.plot(t*24, f, color='red', zorder = 100, linewidth=2.5)

l = np.argsort(phase)



plt.plot(tt[l]*24.,flux[l]-mu[l]+1,'.', color='darkgray', markersize = 10, zorder =1, rasterized=True)
plt.errorbar(xbin*24, ybin+1, yerr=yerr,fmt='o', markersize = 6, color = 'indigo', zorder=100, rasterized=True, linewidth = 3)
plt.xlim(-8.5,8.5)
plt.ylim(top = 1.015)
plt.ylim(.98, 1.015)
plt.ticklabel_format(useOffset=False)
plt.tick_params(labelsize = 30)
plt.minorticks_on()
plt.xlabel('Time From Center of Transit (hours)', fontsize = 30)
plt.ylabel('Normalized Flux', fontsize = 30)

plt.show()

In [ ]:
# example GP plot

In [ ]:
alpha = .75 # transparency
linewidth = 2 # model line
head_length = .003
head_width = .2
arrowLineWidth = 2
color = 'steelblue'

plt.rcParams['lines.linewidth']   = 3
plt.rcParams['axes.linewidth']    = 2
plt.rcParams['xtick.major.width'] = 2
plt.rcParams['ytick.major.width'] = 2
plt.rcParams['ytick.labelsize'] =   15
plt.rcParams['xtick.labelsize'] =   15
plt.rcParams['axes.labelsize'] =    18
plt.rcParams['legend.numpoints'] =  1
plt.rcParams['axes.labelweight']=   'semibold'
plt.rcParams['mathtext.fontset']=   'stix'
plt.rcParams['font.weight'] =       'semibold'
plt.rcParams['axes.titleweight']=   'semibold'
plt.rcParams['axes.titlesize']=     18

plt.rcParams['axes.linewidth']    = 4

plt.rcParams['xtick.minor.width'] = 2
plt.rcParams['ytick.minor.width'] = 2

plt.rcParams['xtick.minor.size'] = 5
plt.rcParams['ytick.minor.size'] = 4 

plt.rcParams['xtick.major.size'] = 6
plt.rcParams['ytick.major.size'] = 5

In [ ]:
t0 = 3235.417319999993
plt.figure(figsize=(14, 6))
plt.plot(time[61671:], flux[61671:], marker='.', linewidth = 0, markersize = 4, color = '.6', alpha=.3, rasterized=True)
plt.plot(time[61671:], model[61671:], 'r-', linewidth = linewidth, alpha = 0.9, rasterized=True)
plt.ylabel('Normalized Flux')
plt.xlabel('BJD-2457000')
plt.xlim(time[61671], time[-1])
plt.ylim(min(flux[61671:])-.01, max(flux[61671:])+.005)
plt.title('Sector 71 Data and GP Model', y=.9, x = .76)

pp = 0
while t0 < time[-1]:
    plt.arrow(t0, min(flux[61671:])-.02, 0, [ 0.075, 0.08,0.06][pp], head_length = head_length, head_width = head_width, linewidth = arrowLineWidth, color = color, rasterized=True)
    t0 += Period
    pp+= 1

plt.minorticks_on()